In [ ]:
#!pip install keras
#!pip install tensorflow
#!pip install plotnine

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from keras.datasets import mnist
import tensorflow.keras as kb
from tensorflow.keras import backend
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier


from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer



from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, ConfusionMatrixDisplay, roc_auc_score, recall_score, precision_score


from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv

import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Senior/Machine Learning/Emotion_classify_Data.csv')
df.head()

Mounted at /content/drive


,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


In [ ]:
df.shape
emotion_counts = df['Emotion'].value_counts()

print(emotion_counts)

Emotion
anger    2000
joy      2000
fear     1937
Name: count, dtype: int64


### Preprocess

In [ ]:

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['Comment'])  # Features
y = df['Emotion']  # Labels


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# build parts of pipeline

column_transformer = make_column_transformer(
    (StandardScaler(), slice(0, -1))
)

model = SVC()

pipeline = make_pipeline(column_transformer, model)

display(pipeline)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  slice(0, -1, None))])),
                ('svc', SVC())])

### Train the SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Define the SVM model
model = SVC()

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],        # Regularization parameter
    'kernel': ['linear', 'rbf'],   # Kernel types
    'gamma': ['scale', 'auto'],    # Kernel coefficient (relevant for 'rbf')
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',            # Optimize for accuracy
    cv=5,                          # 5-fold cross-validation
    verbose=2,                     # Show progress
    n_jobs=-1                      # Use all available cores
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Best Cross-Validation Score: 0.9334581553990351


In [ ]:
# Evaluate performance
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 0.9478114478114478
Classification Report:
               precision    recall  f1-score   support

       anger       0.92      0.96      0.94       392
        fear       0.97      0.92      0.94       416
         joy       0.96      0.97      0.96       380

    accuracy                           0.95      1188
   macro avg       0.95      0.95      0.95      1188
weighted avg       0.95      0.95      0.95      1188



In [ ]:
# Use the best model found by GridSearchCV
best_model = grid_search.best_estimator_

# Make predictions on the full dataset
df['predicted_emotion'] = best_model.predict(X)

# Save the updated DataFrame to a new CSV
df.to_csv("updated_file_with_predictions.csv", index=False)

print("Predictions added and saved to 'updated_file_with_predictions.csv'")


Predictions added and saved to 'updated_file_with_predictions.csv'


In [ ]:
df.head(50)

,Comment,Emotion,predicted_emotion
0,i seriously hate one subject to death but now ...,fear,fear
1,im so full of life i feel appalled,anger,anger
2,i sit here to write i start to dig out my feel...,fear,fear
3,ive been really angry with r and i feel like a...,joy,joy
4,i feel suspicious if there is no one outside l...,fear,fear
5,i feel jealous becasue i wanted that kind of l...,anger,anger
6,when a friend of mine keeps telling me morbid ...,anger,anger
7,i finally fell asleep feeling angry useless an...,anger,anger
8,i feel a bit annoyed and antsy in a good way,anger,anger
9,i feel like i ve regained another vital part o...,joy,joy


In [ ]:
# Filter the DataFrame to show misclassified rows
misclassified = df[df['Emotion'] != df['predicted_emotion']]

# Display the misclassified rows
print(misclassified)

# Optionally, save misclassified rows to a CSV file
misclassified.to_csv("misclassified_rows.csv", index=False)
print("Misclassified rows saved to 'misclassified_rows.csv'")


                                                Comment Emotion  \
49    i was sick with a cold amp not feeling well wo...     joy   
92    im not afraid of going on my own but i feel li...     joy   
93     the possibility of having failed the examination    fear   
204   i know who all think this way so i ve always f...    fear   
228   i still feel like i get walked all over but we...     joy   
...                                                 ...     ...   
5840  i get this strange feeling that even with peop...     joy   
5874  i still feel tortured by feelings or thoughts ...    fear   
5885  i cant get traction and start feeling tortured...    fear   
5886  when my mother was seriously ill and had to be...    fear   
5899  ive been angry and under that anger hurt are n...     joy   

     predicted_emotion  
49               anger  
92                fear  
93                 joy  
204                joy  
228              anger  
...                ...  
5840              fe